# OECM Benchmark Data Pipeline

The Benchmark data pipelines organize and assemble benchmark data needed for the ITR tool.  This pipeline supports the OECM 2023 Benchmark (published 4 Sept 2023).


### Environment variables and dot-env

The following cell looks for a "dot-env" file in some standard locations,
and loads its contents into `os.environ`.

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import osc_ingest_trino as osc

# import python_pachyderm

In [ ]:
import re
import json
from math import log10

Define Environment and Execution Variables

In [ ]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

In [ ]:
# See data-platform-demo/pint-demo.ipynb for quantify/dequantify functions

from pint import Context
from pint_pandas import PintArray
from common_units import ureg

Q_ = ureg.Quantity
PA_ = PintArray

### S3 and boto3 (not used)

### Connecting to Trino with sqlalchemy

In the context of the Data Vault, this pipeline operates with full visibiilty into all the data it prepares for the ITR tool.  When the data is output, it is labeled so that the Data Vault can enforce its data management access rules.

In [ ]:
ingest_catalog = "osc_datacommons_dev"
ingest_schema = "sandbox"
dera_schema = "sandbox"
dera_prefix = "dera_"
gleif_schema = "sandbox"
rmi_schema = "sandbox"
iso3166_schema = "sandbox"
essd_schema = "sandbox"
essd_prefix = "essd_"
demo_schema = "demo_dv"

# engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog)

Tables and their contents:

Annex_Region:
* Electricity Generation
* Transport - Final Energy
* Heating supply and air conditioning
* Installed Capacity
* Final Energy Demand
* Energy-related CO2
* Primary Energy Demand

S_Region (Sector production, Energy Intensity, Carbon Intensity)

Scope_Region:
* Scope 1, 2, 3: GHG and/or CO2
* Scope 1, 2 Production-Centric: GHG and/or CO2

T_region (Carbon budgets: Primary, Secondary, End-Use)

In [ ]:
benchmark_OECM_dir = os.path.abspath("../data/external/OECM 20230904")

In [ ]:
def set_oecm_indexes(df, region, scenario, table):
    try:
        df.columns = pd.Index(["variable"] + df.iloc[0, 1:14].astype("Int64").tolist())
    except ValueError:
        print(
            (
                region,
                table,
            )
        )
        print(df.iloc[0])
        raise
    df["Region"] = region
    df["Scenario"] = scenario
    df["Table"] = table
    return df.set_index(["Region", "Scenario", "Table"]).iloc[2:]

In [ ]:
fossil_fuel_types = [
    "Hard coal (& non-renewable waste)",
    "Hard coal",
    "Lignite",
    "Natural gas",
    "Gas",
    "Gas (w/o H2)",
    "Oil",
    "Diesel",
    "Oil & Diesel",
    "Crude oil",
]

renewable_fuel_types = [
    "Hydro",
    "Wind",
    "Solar",
    "PV",
    "Biomass",
    "Biomass (& renewable waste)",
    "Geothermal",
    "Solar thermal power plants",
    "Ocean energy",
]

In [ ]:
def egen_insert_var(
    df: pd.DataFrame, var_name: str, var_col: str, var_value: float
) -> pd.DataFrame:
    df = df.infer_objects(copy=False).replace(r"[ ]+- ", "", regex=True)
    df.insert(0, "variable", var_name)
    if var_col != "fuel type":
        df[var_col] = df["fuel type"]
        df["fuel type"] = pd.NA
    df.loc[df[var_col] == var_name, var_col] = var_value
    return df


def process_egen(egen_df: pd.DataFrame) -> pd.DataFrame:
    stats_df = egen_df.iloc[53:58]
    res_df = egen_df.iloc[59:62]
    egen_df = egen_df.iloc[:53].copy().rename(columns={"variable": "fuel type"})
    egen_df.insert(1, "fuel category", pd.NA)
    egen_df.insert(2, "producer type", pd.NA)
    powerplants_df = egen_df.iloc[:16].copy()
    powerplants_df = egen_insert_var(
        powerplants_df, "Power plants", "fuel type", "total"
    )
    powerplants_df.loc[
        powerplants_df["fuel type"] == "of which wind offshore", "fuel type"
    ] = "Wind Offshore"
    chp_and_power_df = egen_df.iloc[17:26].copy()
    chp_and_power_df = egen_insert_var(
        chp_and_power_df, "Combined heat and power plants", "fuel type", "total"
    )
    chp_by_producer_df = egen_df.iloc[27:30].copy()
    chp_by_producer_df = egen_insert_var(
        chp_by_producer_df, "CHP by producer", "producer type", "total"
    )
    total_pp_gen_df = egen_df.iloc[31:49].copy()
    total_pp_gen_df = egen_insert_var(
        total_pp_gen_df, "Generation", "Total generation", "total"
    )
    total_pp_gen_df.loc[total_pp_gen_df["fuel type"] == "total", "fuel category"] = (
        "All fuels"
    )
    total_pp_gen_df.loc[
        total_pp_gen_df["fuel type"].isin(fossil_fuel_types).values, "fuel category"
    ] = "Fossil"
    fuel_type_idx = (
        total_pp_gen_df["fuel type"]
        .isin(
            [
                "Fossil",
                "Nuclear",
                "Hydrogen",
                "Renewables (w/o renewable hydrogen)",
            ]
        )
        .values
    )
    total_pp_gen_df.loc[fuel_type_idx, "fuel category"] = total_pp_gen_df.loc[
        fuel_type_idx, "fuel type"
    ]
    total_pp_gen_df.loc[
        total_pp_gen_df["fuel type"] == "of which renewable H2", "fuel category"
    ] = "Renewable H2"
    total_pp_gen_df.loc[
        total_pp_gen_df["fuel type"].isin(renewable_fuel_types).values, "fuel category"
    ] = "Renewables (w/o renewable hydrogen)"
    return pd.concat(
        [
            powerplants_df,
            chp_and_power_df,
            chp_by_producer_df,
            total_pp_gen_df,
            stats_df,
            res_df,
        ]
    )

In [ ]:
def generic_insert_var(df: pd.DataFrame, var_name: str) -> pd.DataFrame:
    df = df.infer_objects(copy=False).replace(r"[ -]+ ", "", regex=True)
    df.insert(0, "variable", var_name)
    df.loc[df["fuel type"] == var_name, "fuel type"] = "total"

In [ ]:
def process_transport(transport_df: pd.DataFrame) -> pd.DataFrame:
    res_df = transport_df.iloc[-2:]
    transport_df = (
        transport_df.iloc[:-3].copy().rename(columns={"variable": "fuel type"})
    )
    transport_df.insert(1, "fuel category", pd.NA)
    transport_df.insert(2, "producer type", pd.NA)
    road_df = transport_df.iloc[:7].copy()
    generic_insert_var(road_df, "road")
    rail_df = transport_df.iloc[8:13].copy()
    generic_insert_var(rail_df, "rail")
    navigation_df = transport_df.iloc[14:18].copy()
    generic_insert_var(navigation_df, "navigation")
    aviation_df = transport_df.iloc[19:23].copy()
    generic_insert_var(aviation_df, "aviation")
    total_df = transport_df.iloc[24:31].copy()
    generic_insert_var(total_df, "total")
    return pd.concat([road_df, rail_df, navigation_df, aviation_df, total_df])

In [ ]:
def process_heat_cool(heat_cool_df: pd.DataFrame) -> pd.DataFrame:
    res_df = heat_cool_df.iloc[-2:]
    heat_cool_df = (
        heat_cool_df.iloc[:-3].copy().rename(columns={"variable": "fuel type"})
    )
    heat_cool_df.insert(1, "fuel category", pd.NA)
    heat_cool_df.insert(2, "producer type", pd.NA)
    district_df = heat_cool_df.iloc[:5].copy()
    generic_insert_var(district_df, "District heating plants")
    chp_heat_df = heat_cool_df.iloc[6:11].copy()
    generic_insert_var(chp_heat_df, "Heat from CHP")
    direct_df = heat_cool_df.iloc[12:20].copy()
    generic_insert_var(direct_df, "Direct heating")
    total_df = heat_cool_df.iloc[21:29].copy()
    generic_insert_var(total_df, "total")
    return pd.concat([district_df, chp_heat_df, direct_df, total_df])

In [ ]:
def process_capacity(capacity_df: pd.DataFrame) -> pd.DataFrame:
    res_df = pd.concat([capacity_df.iloc[-4:-2], capacity_df.iloc[-1:]])
    capacity_df = capacity_df.iloc[:18].copy().rename(columns={"variable": "fuel type"})
    capacity_df.insert(1, "fuel category", pd.NA)
    capacity_df.insert(2, "producer type", pd.NA)
    capacity_df.insert(0, "variable", "Capacity")
    capacity_df = capacity_df.infer_objects(copy=False).replace(
        r"[ ]+- ", "", regex=True
    )
    capacity_df.loc[capacity_df["fuel type"] == "Total generation", "fuel type"] = (
        "total"
    )
    capacity_df.loc[capacity_df["fuel type"] == "total", "fuel category"] = "All fuels"
    capacity_df.loc[
        capacity_df["fuel type"].isin(fossil_fuel_types).values, "fuel category"
    ] = "Fossil"
    fuel_type_idx = (
        capacity_df["fuel type"]
        .isin(
            [
                "Fossil",
                "Nuclear",
                "Hydrogen (fuel cells, gas power plants, gas CHP)",
                "Renewables",
            ]
        )
        .values
    )
    capacity_df.loc[fuel_type_idx, "fuel category"] = capacity_df.loc[
        fuel_type_idx, "fuel type"
    ]
    capacity_df.loc[
        capacity_df["fuel type"] == "of which wind offshore", "fuel category"
    ] = "Wind"
    capacity_df.loc[
        capacity_df["fuel type"] == "of which wind offshore", "fuel type"
    ] = "Wind Offshore"
    capacity_df.loc[
        capacity_df["fuel type"].isin(renewable_fuel_types).values, "fuel category"
    ] = "Renewables"
    return pd.concat([capacity_df, res_df])

In [ ]:
def process_demand(demand_df: pd.DataFrame) -> pd.DataFrame:
    total_df = demand_df.iloc[:2]
    res_df = demand_df.iloc[-7:-5]
    demand_df = demand_df.iloc[:-8].copy().rename(columns={"variable": "fuel type"})
    demand_df.insert(1, "fuel category", pd.NA)
    demand_df.insert(2, "producer type", pd.NA)
    transport_df = demand_df.iloc[2:11].copy()
    generic_insert_var(transport_df, "Transport")
    industry_df = demand_df.iloc[12:25].copy()
    generic_insert_var(industry_df, "Industry")
    other_df = demand_df.iloc[26:39].copy()
    generic_insert_var(other_df, "Other sectors")
    non_energy_df = demand_df.iloc[-4:].copy()
    generic_insert_var(non_energy_df, "Other sectors")
    return pd.concat(
        [total_df, transport_df, industry_df, other_df, res_df, non_energy_df]
    )

In [ ]:
def process_emissions(emissions_df: pd.DataFrame) -> pd.DataFrame:
    population_df = emissions_df.iloc[-2:]
    emissions_df = (
        emissions_df.iloc[:-8].copy().rename(columns={"variable": "fuel type"})
    )
    emissions_df.insert(1, "fuel category", pd.NA)
    emissions_df.insert(2, "producer type", pd.NA)
    condensation_df = emissions_df.iloc[:5].copy()
    generic_insert_var(condensation_df, "Condensation power plants")
    chp_df = emissions_df.iloc[6:11].copy()
    generic_insert_var(chp_df, "Combined heat and power plants")
    CO2e_df = emissions_df.iloc[12:17].copy()
    generic_insert_var(CO2e_df, "CO2 emissions power and CHP plants")
    CO2e_EI_df = emissions_df.iloc[-4:].copy()
    generic_insert_var(CO2e_EI_df, "CO2 intensity (g/kWh)")
    CO2e_sector_df = pd.concat([emissions_df.iloc[24:25], emissions_df.iloc[27:32]])
    generic_insert_var(CO2e_sector_df, "CO2 emissions by sector")
    return pd.concat(
        [condensation_df, chp_df, CO2e_df, CO2e_EI_df, CO2e_sector_df, population_df]
    )

In [ ]:
def process_primary(primary_df: pd.DataFrame) -> pd.DataFrame:
    res_df = primary_df.iloc[-2:]
    primary_df = primary_df.iloc[:14].copy().rename(columns={"variable": "fuel type"})
    primary_df.insert(1, "fuel category", pd.NA)
    primary_df.insert(2, "producer type", pd.NA)
    primary_df.insert(0, "variable", "Primary Energy Demand")
    primary_df = primary_df.infer_objects(copy=False).replace(r"[ ]+- ", "", regex=True)
    primary_df.loc[
        primary_df["fuel type"] == "Total (incl. non-energy-use)", "fuel type"
    ] = "total"
    primary_df.loc[primary_df["fuel type"] == "total", "fuel category"] = "All fuels"
    primary_df.loc[
        primary_df["fuel type"].isin(fossil_fuel_types).values, "fuel category"
    ] = "Fossil"
    fuel_type_idx = (
        primary_df["fuel type"]
        .isin(
            [
                "Fossil (excluding on-energy use)Nuclear",
                "Renewables",
            ]
        )
        .values
    )
    primary_df.loc[fuel_type_idx, "fuel category"] = primary_df.loc[
        fuel_type_idx, "fuel type"
    ]
    primary_df.loc[
        primary_df["fuel type"].isin(renewable_fuel_types).values, "fuel category"
    ] = "Renewables"
    return pd.concat([primary_df, res_df])

In [ ]:
egen_df_list = []
transport_df_list = []
heat_cool_df_list = []
capacity_df_list = []
demand_df_list = []
emissions_df_list = []
primary_df_list = []


def process_annex_country(oecm_dir):
    data_dir = os.path.join(oecm_dir, "Annex_Country")
    for file in [
        f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))
    ]:
        if file.startswith("~") or not file.endswith(".xlsx"):
            continue
        try:
            raw_df = pd.read_excel(os.path.join(data_dir, file))
        except ValueError:
            print(file)
            raise
        region = raw_df.columns[1]
        temperature = Q_(raw_df.columns[8])
        p = re.compile(r"(.*) \[(.*)\]")
        m = p.match(raw_df.iloc[0, 1])
        egen_title, egen_units = m.groups()
        egen_df = raw_df.iloc[1:65, 1:15]
        egen_df = set_oecm_indexes(egen_df, region, temperature, egen_title)
        egen_df = process_egen(egen_df)
        egen_df_list.append(egen_df)
        m = p.match(raw_df.iloc[66, 1])
        transport_title, transport_units = m.groups()
        transport_df = raw_df.iloc[67:103, 1:15]
        transport_df = set_oecm_indexes(
            transport_df, region, temperature, transport_title
        )
        transport_df = process_transport(transport_df)
        transport_df_list.append(transport_df)
        m = p.match(raw_df.iloc[104, 1])
        heat_cool_title, heat_cool_units = m.groups()
        heat_cool_df = raw_df.iloc[106:140, 1:15]
        heat_cool_df = set_oecm_indexes(
            heat_cool_df, region, temperature, heat_cool_title
        )
        heat_cool_df = process_heat_cool(heat_cool_df)
        heat_cool_df_list.append(heat_cool_df)
        m = p.match(raw_df.iloc[0, 16])
        capacity_title, capacity_units = m.groups()
        capacity_df = raw_df.iloc[1:26, 16:30]
        capacity_df = set_oecm_indexes(capacity_df, region, temperature, capacity_title)
        capacity_df = process_capacity(capacity_df)
        capacity_df_list.append(capacity_df)
        m = p.match(raw_df.iloc[27, 16])
        demand_title, demand_units = m.groups()
        demand_df = raw_df.iloc[28:77, 16:30]
        demand_df = set_oecm_indexes(demand_df, region, temperature, demand_title)
        demand_df = process_demand(demand_df)
        demand_df_list.append(demand_df)
        m = p.match(raw_df.iloc[78, 16])
        emissions_title, emissions_units = m.groups()
        emissions_df = raw_df.iloc[79:116, 16:30]
        emissions_df = set_oecm_indexes(
            emissions_df, region, temperature, emissions_title
        )
        emissions_df = process_emissions(emissions_df)
        emissions_df_list.append(emissions_df)
        m = p.match(raw_df.iloc[117, 16])
        primary_title, primary_units = m.groups()
        primary_df = raw_df.iloc[118:141, 16:30]
        primary_df = set_oecm_indexes(primary_df, region, temperature, primary_title)
        primary_df = process_primary(primary_df)
        primary_df_list.append(primary_df)
    annex_egen_df = pd.concat(egen_df_list)
    annex_transport_df = pd.concat(transport_df_list)
    annex_heat_cool_df = pd.concat(heat_cool_df_list)
    annex_capacity_df = pd.concat(capacity_df_list)
    annex_demand_df = pd.concat(demand_df_list)
    annex_emissions_df = pd.concat(emissions_df_list)
    annex_primary_df = pd.concat(primary_df_list)
    return (
        (
            egen_units,
            annex_egen_df,
        ),
        (
            transport_units,
            annex_transport_df,
        ),
        (
            heat_cool_units,
            annex_heat_cool_df,
        ),
        (
            capacity_units,
            annex_capacity_df,
        ),
        (
            demand_units,
            annex_demand_df,
        ),
        (
            emissions_units,
            annex_emissions_df,
        ),
        (
            primary_units,
            annex_primary_df,
        ),
    )

In [ ]:
(
    (
        annex_egen_units,
        annex_egen_df,
    ),
    (
        annex_transport_units,
        annex_transport_df,
    ),
    (
        annex_heat_cool_units,
        annex_heat_cool_df,
    ),
    (
        annex_capacity_units,
        annex_capacity_df,
    ),
    (
        annex_demand_units,
        annex_demand_df,
    ),
    (
        annex_emissions_units,
        annex_emissions_df,
    ),
    (
        annex_primary_units,
        annex_primary_df,
    ),
) = process_annex_country(benchmark_OECM_dir)

In [ ]:
def fix_units(df: pd.DataFrame) -> None:
    df.Unit = df.Unit.replace(r"[\[\]]", "", regex=True)
    df.Unit = df.Unit.replace(r"(.*) */ *([^ ]+ .+)", r"\1/(\2)", regex=True)
    df.Unit = df.Unit.replace(
        [
            r"t ?CO2 (equivalent|equiv\.)",
            r"p(erson)? km",
            r"steel",
            r"cement",
            r"clinker",
            r"alu",
            r" GDP",
            r"bn \$$",
        ],
        ["tCO2e", "pkm", "Steel", "Cement", "Clinker", "Aluminium", "GDP", "bn $GDP"],
        regex=True,
    )

In [ ]:
S_USA_values = None


def process_S_CSV(oecm_dir: str) -> pd.DataFrame:
    global S_USA_df
    data_dir = os.path.join(oecm_dir, "Sector_CSV")
    sector_df_list = []

    for file in sorted(
        [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))],
        reverse=True,
    ):
        if file.startswith("~") or not file.endswith(".csv"):
            continue
        try:
            raw_df = pd.read_csv(os.path.join(data_dir, file)).iloc[:, 1:]
        except ValueError:
            print(file)
            raise
        region = file[2:-4]
        raw_df["Region"] = region
        fix_units(raw_df)
        if region == "USA":
            S_USA_values = raw_df.Sector.values
        elif region == "Australia":
            raw_df.loc[
                raw_df.Subsector.eq("Cement - production volume"), "Subsector"
            ] = "Cement - production volume in mega tonnes per year"
            raw_df.loc[
                raw_df.Subsector.eq(
                    "Clinker - production volume (based on clinker to cement ratio)"
                ),
                "Subsector",
            ] = "Clinker - production volume in mega tonnes per year"
            raw_df = raw_df[raw_df.Subsector.notna()]
        if len(raw_df.Sector.unique()) == 1:
            fish_ser = raw_df.Subsector.str.contains(r"fish", case=False, regex=True)
            first = fish_ser.idxmax()
            if first:
                last = fish_ser[::-1].idxmax()
                raw_df = pd.concat([raw_df.iloc[:first], raw_df.iloc[last + 1 :]])
            raw_df = raw_df[raw_df["2017"] != 2017]
            raw_df.Sector = S_USA_values
        raw_df.loc[
            (raw_df.Sector == "Power Generation") & (raw_df.Data == "Energy Intensity"),
            "Data",
        ] = "Production"
        # raw_df.loc[raw_df.Sector.isin(["Aviation", "Shipping", "Road"]) & (raw_df.Data=="Quantity of Service"), "Data"] = "Production"
        # raw_df.loc[(raw_df.Sector=="Chemical Industries") & (raw_df.Data=="Market"), "Data"] = "Production"
        # raw_df.loc[(raw_df.Sector=="Residential & Commercial Buildings and Construction") & (raw_df.Data=="Stock"), "Data"] = "Production"
        # raw_df.loc[raw_df.Data=="Market Development", "Data"] = "Production"
        raw_df.loc[
            raw_df.Data.isin(["Quantity of Service", "Market", "Market Development"]),
            "Data",
        ] = "Production"
        raw_df.set_index(["Sector", "Region", "Data", "Subsector"], inplace=True)
        raw_df.sort_index(inplace=True)
        try:
            raw_df.loc[
                (
                    "Total Materials / Steel",
                    slice(None),
                    "Production",
                    "Regional: Minning iron ore - production volume in mega tonnes per year",
                ),
                "Unit",
            ] = "Mt Iron/a"
            raw_df.loc[
                (
                    "Total Materials / Steel",
                    slice(None),
                    "Production",
                    "Annual production volume- Iron & Steel Industry",
                ),
                "Unit",
            ] = "Mt Steel/a"
        except KeyError:
            print(file)
        raw_df.loc[
            (
                "Aluminium",
                slice(None),
                "Production",
                "Minning bauxite - production volume in mega tonnes per year",
            ),
            "Unit",
        ] = "Mt Bauxite/a"
        raw_df.loc[
            (
                "Aluminium",
                slice(None),
                "Production",
                "Annual production volume- aluminium Industry",
            ),
            "Unit",
        ] = "Mt Aluminum/a"
        raw_df.loc[
            (
                "Total Materials / Cement",
                slice(None),
                "Production",
                "Cement - production volume in mega tonnes per year",
            ),
            "Unit",
        ] = "Mt Cement/a"
        raw_df.loc[
            (
                "Total Materials / Cement",
                slice(None),
                "Production",
                "Clinker - production volume in mega tonnes per year",
            ),
            "Unit",
        ] = "Mt Clinker/a"
        raw_df.set_index(["Unit"], append=True, inplace=True)
        raw_df.columns = raw_df.columns.astype("Int64")
        new_index = range(2017, 2051)
        with warnings.catch_warnings():
            warnings.filterwarnings("error")
            try:
                raw_df = raw_df.apply(
                    lambda x: x.reindex(new_index).interpolate(), axis=1
                )
            except FutureWarning:
                print(raw_df.droplevel(level=[1, 2, 3, 4]))
                print(raw_df.dtypes)
                print(raw_df[2019])
        sector_df_list.append(raw_df)
        if file == "S_USA.csv":
            S_USA_df = raw_df
    return pd.concat(sector_df_list)

In [ ]:
sector_df = process_S_CSV(benchmark_OECM_dir)
sector_df

In [ ]:
def process_Scope_CSV(oecm_dir: str) -> pd.DataFrame:
    data_dir = os.path.join(oecm_dir, "Scope_CSV")
    scope_df_list = []

    for file in [
        f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))
    ]:
        if file.startswith("~") or not file.endswith(".csv"):
            continue
        try:
            raw_df = pd.read_csv(
                os.path.join(data_dir, file), dtype={"Remarks": "string"}
            ).iloc[:, 1:]
        except ValueError:
            print(file)
            raise
        fix_units(raw_df)
        raw_df.Unit = raw_df.Unit.replace(r"[\[\]]", "", regex=True)
        raw_df.Unit = raw_df.Unit.replace(r"CO2 equiv.", "CO2e", regex=True)
        # Fix errant "0" and other errors in Transport Units
        raw_df.loc[raw_df.Label.str.contains("GHG (CO2 only)", regex=False), "Unit"] = (
            "Mt CO2/a"
        )
        raw_df.Sector = raw_df.Sector.replace(r" +- *$", "", regex=True)
        raw_df.Subsector = raw_df.Subsector.replace(r" +- *$", "", regex=True)
        raw_df.Subsector = raw_df.Subsector.replace(r" +$", "", regex=True)
        raw_df.Subsector = raw_df.Subsector.replace(
            r"&Tobacco", "& Tobacco", regex=True
        )
        raw_df.Description = raw_df.Description.replace(
            r"&Tobacco", "& Tobacco", regex=True
        )
        raw_df["Energy only"] = raw_df.Description.map(lambda x: "(energy-only)" in x)
        raw_df.set_index(
            [
                "Sector",
                "Subsector",
                "Scope",
                "Label",
                "Energy only",
                "Description",
                "Unit",
                "Remarks",
            ],
            inplace=True,
        )
        raw_df.columns = raw_df.columns.astype("Int64")
        new_index = range(2017, 2051)
        raw_df = raw_df.apply(lambda x: x.reindex(new_index).interpolate(), axis=1)
        # Remove '_Results_all.csv' while inserting region/country name
        raw_df.insert(0, "Region", file[0:-16])
        scope_df_list.append(raw_df)
    scope_df = pd.concat(scope_df_list)
    scope_df.reset_index(["Description"], inplace=True)
    scope_df.set_index("Region", append=True, inplace=True)
    scope_df = scope_df.reorder_levels(
        [
            "Sector",
            "Subsector",
            "Region",
            "Scope",
            "Label",
            "Energy only",
            "Unit",
            "Remarks",
        ]
    )
    return scope_df

In [ ]:
scope_df = process_Scope_CSV(benchmark_OECM_dir)
scope_df

In [ ]:
sector_df[sector_df.index.get_level_values("Data") == "Production"].head(100)

In [ ]:
pd.options.display.max_rows = 250

In [ ]:
def process_T_country(oecm_dir):
    data_dir = os.path.join(oecm_dir, "T_Country")
    sector_df_list = []
    sector_sum_df_list = []
    sector_share_df_list = []
    energy_df_list = []
    energy_sum_df_list = []
    energy_share_df_list = []

    for file in [
        f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))
    ]:
        if file.startswith("~") or not file.endswith(".xlsx"):
            continue
        try:
            raw_df = pd.read_excel(os.path.join(data_dir, file)).iloc[:, 1:]
        except ValueError:
            print(file)
            raise
        if file in "T_India.xlsx":
            energy_start_row = 22
        else:
            energy_start_row = 21
        region = raw_df.columns[0]
        # Australia is missing its country label
        if region.startswith("Unnamed: "):
            region = file[2:-5]
        title = raw_df.iat[0, 0]
        raw_df.iloc[:, 2] = raw_df.iloc[:, 2].replace(r"[\[\]]", "", regex=True)
        sector_df = raw_df.iloc[2:19, :47]
        try:
            sector_df.columns = pd.Index(
                ["Sector", "Industry", "Units", "Delay"]
            ).append(pd.Index(raw_df.iloc[1, 4:47].astype("Int64")))
        except (ValueError, TypeError):
            print(file)
            break
        sector_df.set_index(sector_df.columns[0:4].to_list(), inplace=True)
        sector_df["Region"] = region
        sector_df.set_index(["Region", sector_df.index], inplace=True)
        sector_sum_df = raw_df.iloc[2:19, 47:51]
        sector_sum_df.index = sector_df.index
        sector_sum_df.columns = raw_df.iloc[1, 47:51].values
        sector_share_df = raw_df.iloc[2:19, 51:52]
        sector_share_df.index = sector_df.index
        sector_share_df.columns = raw_df.iloc[0, 51:52].values
        energy_df = raw_df.iloc[energy_start_row : energy_start_row + 8, :47]
        try:
            energy_df.columns = pd.Index(
                ["Energy services", "Fossil Fuels", "Units", "Delay"]
            ).append(pd.Index(raw_df.iloc[energy_start_row, 4:47].astype("Int64")))
        except TypeError:
            print(file)
            break
        energy_df.set_index(energy_df.columns[0:4].to_list(), inplace=True)
        energy_df["Region"] = region
        energy_df.set_index(["Region", energy_df.index], inplace=True)
        energy_sum_df = raw_df.iloc[energy_start_row : energy_start_row + 8, 47:51]
        energy_sum_df.index = energy_df.index
        energy_sum_df.columns = raw_df.iloc[energy_start_row, 47:51].values
        energy_share_df = raw_df.iloc[energy_start_row : energy_start_row + 8, 51:52]
        energy_share_df.index = energy_df.index
        energy_share_df.columns = raw_df.iloc[energy_start_row, 51:52].values

        sector_df_list.append(sector_df)
        sector_sum_df_list.append(sector_sum_df)
        sector_share_df_list.append(sector_share_df)
        energy_df_list.append(energy_df)
        energy_sum_df_list.append(energy_sum_df)
        energy_share_df_list.append(energy_share_df)

    return (
        pd.concat(sector_df_list),
        pd.concat(sector_sum_df_list),
        pd.concat(sector_share_df_list),
        pd.concat(energy_df_list),
        pd.concat(energy_sum_df_list),
        pd.concat(energy_share_df_list),
    )

In [ ]:
(
    T_sector_df,
    T_sector_sum_df,
    T_sector_share_df,
    energy_df,
    energy_sum_df,
    energy_share_df,
) = process_T_country(benchmark_OECM_dir)

### Definitions and dictionaries for reading from / writing to the outside world

In [ ]:
transport_elements = [
    "Subsector",
    "Total CO2 Emissions",
    "Emission Intensity",
    "Energy Intensity",
]
bldgs_elements = [
    "Parameter",
    "Residential Buildings",
    "Commercial Buildings",
    "Construction: Residential and Commercial Building - Economic value",
]

benchmark_years = pd.Series(
    name="Production", index=pd.Index(list(range(2017, 2051))), dtype="float64"
)
benchmark_years.index.name = "Year"

# Maps Sector (really Sub-Sector) to Sheet data
oecm_dict = {
    # Subsector: Parameter / Subsector tag; Sheet; Aggregates as; Aggregates to; CO2 label; Production Units; Intensity Units
    "Materials / Steel": [
        "Parameter",
        "Steel",
        "Materials / Steel",
        "Annual production volume- Iron & Steel Industry",
        "Total CO2 equivalent",
        "Mt Steel",
        "t CO2e/(t Steel)",
    ],
    "Power Utilities": [
        "Subsector",
        "Utilities",
        "Power Utilities",
        "Total public power generation (incl. CHP, excluding auto producers, losses)",
        "Total CO2 equivalent",
        "TWh",
        "t CO2e/MWh",
    ],
    "Gas Utilities": [
        "Subsector",
        "Utilities",
        "Gas Utilities",
        "Total Energy transport & distribution (gas, synthetic fuels & hydrogen)",
        "Total CO2 equivalent",
        "PJ",
        "t CO2e/GJ",
    ],
    "Utilities": [
        "Subsector",
        "Utilities",
        "Utilities",
        "Total Energy Production (power + gas/fuels)",
        "Total CO2 equivalent",
        "PJ",
        "t CO2e/GJ",
    ],
    "Coal": [
        "Subsector",
        "Energy",
        "Coal",
        "Coal: Gross Production (for regional energy demand - incl. non-energy-use)",
        "",
        "Mt Coal",
        "t CO2e/(t Coal)",
    ],
    # Note we have to convert from /d to /a
    "Oil": [
        "Subsector",
        "Energy",
        "Oil",
        "Oil: Gross Production (for regional energy demand - incl. non-energy-use)",
        "",
        "MMbbl/d",
        "t CO2e/Mbbl",
    ],
    "Gas": [
        "Subsector",
        "Energy",
        "Gas",
        "Gas: Gross Production (for regional energy demand - incl. non-energy-use)",
        "",
        "bcm CH4",
        "Mt CO2e/(bcm CH4)",
    ],
    "Energy Industry": [
        "Subsector",
        "Energy",
        "Energy Industry",
        "Total Energy Production - Energy, Gas, Oil &Coal Sector",
        "Total CO2 equivalent",
        "PJ",
        "t CO2e/GJ",
    ],
    "Road: LDV / Passenger Transport": [
        "Subsector",
        "Transport_UNPRI",
        "Road Transport",
        "Road Transport (excluding vehicle manufacturing)",
        "",
        "pkm",
        "g CO2e/pkm",
    ],
    "Road: Trucks / Freight Transport": [
        "Subsector",
        "Transport_UNPRI",
        "Road Transport",
        "Road Transport (excluding vehicle manufacturing)",
        "",
        "tkm",
        "g CO2e/tkm",
    ],
    "Aluminium Industry": [
        "Parameter",
        "Alu",
        "Aluminium Industry",
        "Annual production volume- aluminium Industry",
        "Total CO2 equivalent",
        "Mt Aluminum",
        "t CO2e/(t Aluminum)",
    ],
    "Materials / Cement": [
        "Parameter",
        "Cement",
        "Materials / Cement",
        "Cement - production volume in mega tonnes per year",
        "Total CO2 equivalent",
        "Mt Cement",
        "t CO2e/(t Cement)",
    ],
    "Construction Buildings": [
        "Parameter",
        "Buildings",
        "Construction Buildings",
        "Construction: Residential and Commercial Building - Economic value",
        "Total CO2 equivalent",
        "billion USD",
        "t CO2e/(million USD)",
    ],
    "Residential Buildings": [
        "Parameter",
        "Buildings",
        "Residential Buildings",
        "Residential Buildings",
        "Total CO2 equivalent",
        "billion m**2",
        "t CO2e/(million m**2)",
    ],
    "Commercial Buildings": [
        "Parameter",
        "Buildings",
        "Commercial Buildings",
        "Commercial Buildings",
        "Total CO2 equivalent",
        "billion m**2",
        "t CO2e/(million m**2)",
    ],
    "Chemical Industry": [
        "Parameter",
        "Chemical Industry",
        "Chemical Industry",
        "Total Chemical Industry",
        "Total CO2 equivalent",
        "billion USD",
        "kg CO2e/USD",
    ],
    "Pharmaceutical Industry": [
        "Parameter",
        "Chemical Industry",
        "Pharmaceutical Industry   -",
        "Pharmaceutical Industry - Economic value",
        "",
        "billion USD",
        "kg CO2e/USD",
    ],
    "Agricultural Chemicals": [
        "Parameter",
        "Chemical Industry",
        "Agricultural Chemicals   -",
        "Agricultural Chemicals - Economic value",
        "",
        "billion USD",
        "kg CO2e/USD",
    ],
    "Inorganic Chemicals and Consumer Products": [
        "Parameter",
        "Chemical Industry",
        "Inorganic Chemicals and Consumer Products   -",
        "Inorganic Chemicals and Consumer Products - Economic value",
        "",
        "billion USD",
        "kg CO2e/USD",
    ],
    "Manufactured Fibres & Synthetic Rubber": [
        "Parameter",
        "Chemical Industry",
        "Manufactured Fibres & Synthetic Rubber   -",
        "Manufactured Fibres & Synthetic Rubber - Economic value",
        "",
        "billion USD",
        "kg CO2e/USD",
    ],
    "Bulk Petrochemicals & Intermediates, Plastic Resins": [
        "Parameter",
        "Chemical Industry",
        "Bulk Petrochemicals & Intermediates, Plastic Resins   -",
        "Bulk Petrochemicals & Intermediates, Plastic Resins - Economic value",
        "",
        "billion USD",
        "kg CO2e/USD",
    ],
    "Textile & Leather": [
        "Parameter",
        "Tex & Lea",
        "Textile & Leather",
        "Total Textile & Leather",
        "Total CO2 equivalent",
        "billion USD",
        "kg CO2e/USD",
    ],
}

# From OECM (Sub-)Sector name to ITR Sector Name.  Keys MUST BE UNIQUE
itr_dict = {
    "Materials / Steel": "Steel",
    "Power Utilities": "Electricity Utilities",
    "Gas Utilities": "Gas Utilities",
    "Utilities": "Utilities",
    "Coal": "Coal",
    "Oil": "Oil",
    "Gas": "Gas",
    "Energy Industry": "Energy",
    "Road: LDV / Passenger Transport": "Autos",
    "Road: Trucks / Freight Transport": "Trucking",
    "Aluminium Industry": "Aluminum",
    "Materials / Cement": "Cement",
    "Construction Buildings": "Construction Buildings",
    "Residential Buildings": "Residential Buildings",
    "Commercial Buildings": "Commercial Buildings",
    "Chemical Industry": "Chemicals",
    "Pharmaceutical Industry": "Pharmaceuticals",
    "Agricultural Chemicals": "Ag Chem",
    "Inorganic Chemicals and Consumer Products": "Consumer Products",
    "Manufactured Fibres & Synthetic Rubber": "Fiber & Rubber",
    "Bulk Petrochemicals & Intermediates, Plastic Resins": "Petrochem & Plastics",
    "Textile & Leather": "Textiles",
}

### Construct JSON benchmark structures

1.  Load Regional Workbook
2.  Process each Sector in the Workbook
3.  Convert resulting dataframe to dictionary structure
4.  Merge each Region/Sector dictionary into main benchmark dictionary

Note that we use linear interpolation when the overall interpolation is more than a 2:1 ratio start to finish
CAGR gets wonky both as the endpoint approaches zero (ratio becomes infinite); but it's also funky when slope is steep (though not infinitely steep)

In [ ]:
bm_seed = {
    "benchmark_temperature": "1.5 delta_degC",
    "benchmark_global_budget": "396 Gt CO2",
    "is_AFOLU_included": False,
}

production_bm = bm_seed
# OECM defines both 'OECM' and 'Production-Centric' benchmarks
ei_bms = [bm_seed.copy(), bm_seed.copy()]

region_dict = {
    "Global": "OECM_Global_2023_04_22_Results",
    "Europe": "OECM_OECD_Europe_2022_04_22_results",
    "North America": "OECM_OECD_North_America_2022_04_22_results_0",
}


def merge_bm_dicts(main, new):
    for scope in new.keys():
        if not main.get(scope):
            main[scope] = new[scope]
            continue
        main[scope]["benchmarks"].append(new[scope]["benchmarks"][0])


benchmark_OECM_dir = os.path.abspath("../data/external/OECM 20230904")

oil_and_gas_dict = {}

for subsector, sector_elements in oecm_dict.items():
    sheet = sector_elements[1]
    ei_unit = sector_elements[6]
    continue
    for region, filename in region_dict.items():
        df = pd.read_excel(
            f"{benchmark_OECM_dir}/{filename}.xlsx", sheet_name=sheet, dtype=str
        )
        orig_df = df.applymap(lambda x: x.rstrip(), na_action="ignore")
        print(f"Sector {subsector} Region {region}")

        for production_centric in [True, False]:
            df = process_sector_benchmark(
                orig_df, subsector, region, sector_elements, production_centric
            )
            if subsector in ["Oil", "Gas"]:
                oil_and_gas_dict[(subsector, region, production_centric)] = df
            # It's tempting to concatenate these DataFrames, but doing so wrecks the nice PintArrays created for Production and EI
            # So instead, build up the respective dictionaries with each dataframe we process

            bm_ei_scopes = {
                scope: {
                    "production_centric": production_centric,
                    "benchmarks": [
                        {
                            "sector": itr_dict[subsector],
                            "region": region,
                            "benchmark_metric": ei_unit,
                            "scenario name": "OECM 1.5 Degrees",
                            "release date": "2022",
                            "projections_nounits": [
                                {"year": year, "value": value.m}
                                for year, value in zip(df.index, df[f"EI_{scope}"])
                            ],
                        }
                    ],
                }
                for scope in ["S1", "S2", "S1S2", "S1S2S3"]
                if f"EI_{scope}" in df.columns
            }

            if "EI_S3" in df.columns:
                bm_ei_scopes["S3"] = {
                    "production_centric": production_centric,
                    "benchmarks": [
                        {
                            "sector": itr_dict[subsector],
                            "region": region,
                            "benchmark_metric": ei_unit,
                            "scenario name": "OECM 1.5 Degrees",
                            "release date": "2022",
                            "projections_nounits": [
                                {"year": year, "value": value.m}
                                for year, value in zip(df.index, df.EI_S3)
                            ],
                        }
                    ],
                }

            merge_bm_dicts(ei_bms[production_centric], bm_ei_scopes)

        # Production is not conditioned on scope--we shouldn't even need it!  It's also not dependent on "Production-centric"
        new_prod_bm = {
            scope: {
                "benchmarks": [
                    {
                        "sector": itr_dict[subsector],
                        "region": region,
                        "benchmark_metric": "dimensionless",
                        "scenario name": "OECM 1.5 Degrees",
                        "release date": "2022",
                        "base_year_production": str(df.Production.values[0]),
                        "projections_nounits": [
                            {"year": year, "value": value}
                            for year, value in zip(df.index, df.d_Production)
                        ],
                    }
                ]
            }
            for scope in ["AnyScope"]
        }
        merge_bm_dicts(production_bm, new_prod_bm)

### Oil & Gas

OECM separates `Energy` into `Coal`, `Oil`, and `Gas`.  But many users of the benchmark characterize companies as `Oil & Gas` companies, which is consistent with other benchmarks.  We synthesize an `Oil & Gas` sector for OECM here.

In [ ]:
# From ITR.data.osc_units
# ureg.define("boe = 5.712 GJ")
ureg.define("boe = 6.1178632 GJ = BoE")
ureg.define("Mbbl = 1e3 bbl")
ureg.define("MMbbl = 1e6 bbl")
ureg.define("scf = ft**3")
ureg.define("mscf = 1000 scf = Mscf")
ureg.define("mmscf = 1000000 scf = MMscf")

oil = Context("oil")
oil.add_transformation(
    "[carbon] * [mass] ** 2 / [length] / [time] ** 2",
    "[carbon] * [mass]",
    lambda ureg, x: x * ureg("bbl/boe").to_reduced_units(),
)
# oil.add_transformation('boe', 'kg CO2e', lambda ureg, x: x * ureg('431.87 kg CO2e / boe')
oil.add_transformation("bbl", "boe", lambda ureg, x: x * ureg("boe") / ureg("bbl"))
oil.add_transformation("boe", "bbl", lambda ureg, x: x * ureg("bbl") / ureg("boe"))
# Converting intensity t CO2/bbl -> t CO2/boe
oil.add_transformation(
    "[carbon] * [mass] / [length] ** 3",
    "[carbon] * [time] ** 2 / [length] ** 2",
    lambda ureg, x: (x * ureg("bbl/boe")).to_reduced_units(),
)
oil.add_transformation(
    "[carbon] * [time] ** 2 / [length] ** 2",
    "[carbon] * [mass] / [length] ** 3",
    lambda ureg, x: (x * ureg("boe/bbl")).to_reduced_units(),
)
ureg.add_context(oil)

ureg.define("bcm = 1000000000 m**3")
NG_DENS = 0.7046 * ureg("kg CH4/(m**3 CH4)")  # 0.657
NG_SE = 54.84 * ureg("MJ/(kg CH4)")  # specific energy (energy per mass); range is 50-55
ng = Context("ngas")
ng.add_transformation("[volume] CH4", "[mass] CH4", lambda ureg, x: x * NG_DENS)
ng.add_transformation("[mass] CH4", "[volume] CH4", lambda ureg, x: x / NG_DENS)
ng.add_transformation("[volume] CH4 ", "[energy]", lambda ureg, x: x * NG_DENS * NG_SE)
ng.add_transformation("[energy]", "[volume] CH4", lambda ureg, x: x / (NG_DENS * NG_SE))
ng.add_transformation(
    "[carbon] * [length] * [methane] * [time] ** 2",
    "[carbon] * [mass]",
    lambda ureg, x: x * NG_DENS * NG_SE,
)
ng.add_transformation(
    "[carbon] * [mass] / [volume] / [methane]",
    "[carbon] * [mass] / [energy]",
    lambda ureg, x: x / (NG_DENS * NG_SE),
)
ng.add_transformation(
    "[carbon] * [time] ** 2 / [length] ** 2",
    "[carbon] * [mass] / [length] ** 3 / [methane]",
    lambda ureg, x: x * NG_DENS * NG_SE,
)

# Cannot convert from 'megawatt_hour / CH4 / mmscf' ([mass] / [length] / [methane] / [time] ** 2) to 'dimensionless' (dimensionless)
# conversion to dimensionless throws key error on '' in ureg

ng.add_transformation(
    "Mscf CH4", "kg CO2e", lambda ureg, x: x * ureg("54.87 kg CO2e / (Mscf CH4)")
)
ng.add_transformation(
    "g CH4", "g CO2e", lambda ureg, x: x * ureg("44 g CO2e / (16 g CH4)")
)
ureg.add_context(ng)

ureg.enable_contexts("oil", "ngas")

In [ ]:
for production_centric in [True, False]:
    break
    df_all = pd.concat(
        [v for k, v in oil_and_gas_dict.items() if k[2] is production_centric]
    )
    for region in df_all.Region.unique():
        df = df_all[df_all.Region == region]
        df_oil = df[df.Sector == "Oil"].copy()
        df_oil.Production = df_oil.Production.astype(
            f"pint[{str(df_oil.Production.values[0].u)}]"
        )
        df_gas = df[df.Sector == "Gas"].copy()
        df_gas.Production = df_gas.Production.astype(
            f"pint[{str(df_gas.Production.values[0].u)}]"
        )
        o_and_g_em_unit = "t CO2e"
        o_and_g_prod_unit = "TJ"
        o_and_g_bm_unit = f"{o_and_g_em_unit}/{o_and_g_prod_unit}"
        bm_ei_scopes = {
            scope: {
                "production_centric": production_centric,
                "benchmarks": [
                    {
                        "sector": "Oil & Gas",
                        "region": region,
                        "benchmark_metric": o_and_g_bm_unit,
                        "scenario name": "OECM 1.5 Degrees",
                        "release date": "2022",
                        "projections_nounits": [
                            {
                                "year": year,
                                "value": (
                                    oil_em.m_as(o_and_g_em_unit)
                                    + gas_em.m_as(o_and_g_em_unit)
                                )
                                / (
                                    oil_prod.m_as(o_and_g_prod_unit)
                                    + gas_prod.m_as(o_and_g_prod_unit)
                                ),
                            }
                            for year, oil_em, oil_prod, gas_em, gas_prod in zip(
                                df_oil.index,
                                df_oil[scope],
                                df_oil.Production,
                                df_gas[scope],
                                df_gas.Production,
                            )
                        ],
                    }
                ],
            }
            for scope in ["S1", "S2", "S1S2", "S1S2S3"]
            if f"EI_{scope}" in df.columns
        }

        if "EI_S3" in df.columns:
            bm_ei_scopes["S3"] = {
                "production_centric": production_centric,
                "benchmarks": [
                    {
                        "sector": "Oil & Gas",
                        "region": region,
                        "benchmark_metric": o_and_g_bm_unit,
                        "scenario name": "OECM 1.5 Degrees",
                        "release date": "2022",
                        "projections_nounits": [
                            {
                                "year": year,
                                "value": (
                                    oil_em.m_as(o_and_g_em_unit)
                                    + gas_em.m_as(o_and_g_em_unit)
                                )
                                / (
                                    oil_prod.m_as(o_and_g_prod_unit)
                                    + gas_prod.m_as(o_and_g_prod_unit)
                                ),
                            }
                            for year, oil_em, oil_prod, gas_em, gas_prod in zip(
                                df_oil.index,
                                df_oil["S3"],
                                df_oil.Production,
                                df_gas["S3"],
                                df_gas.Production,
                            )
                        ],
                    }
                ],
            }

        merge_bm_dicts(ei_bms[production_centric], bm_ei_scopes)

        # Production is not conditioned on scope--we shouldn't even need it!  It's also not dependent on "Production-centric", despite appearances
        # Alas, we have to re-synthesize the year-over-year growth rate based on the sum PJ of the two components

        if not production_centric:
            base_prod = df_oil.Production.values[0].m_as(
                "PJ"
            ) + df_gas.Production.values[0].m_as("PJ")
            prod_series = (
                df_oil.Production.pint.m_as("PJ").add(df_gas.Production.pint.m_as("PJ"))
                / base_prod
            )
            prod_delta = prod_series.div(prod_series.shift(1))
            prod_delta.iloc[0] = 1.0
            prod_delta = prod_delta.sub(1.0)
            new_prod_bm = {
                scope: {
                    "benchmarks": [
                        {
                            "sector": "Oil & Gas",
                            "region": region,
                            "benchmark_metric": "dimensionless",
                            "scenario name": "OECM 1.5 Degrees",
                            "release date": "2022",
                            "base_year_production": f"{base_prod} PJ",
                            "projections_nounits": [
                                {"year": year, "value": value}
                                for year, value in zip(
                                    prod_delta.index, prod_delta.values
                                )
                            ],
                        }
                    ]
                }
                for scope in ["AnyScope"]
            }
            merge_bm_dicts(production_bm, new_prod_bm)

### Emit Sector Benchmark Data

In [ ]:
# https://til.simonwillison.net/python/json-floating-point
# Modified to blend the concept of "precision after the decimal point" with "significant figures" (SF).
# For numbers in (-1,1), gives PRECISION=3 sig figs.  For numbers outside that range, but within (-10,10), an addition SF.
# Will provide up to PRECISION-1 additional SFs (default 2) for larger absolute magnitudes.


# from math import log10
def round_floats(o, precision=3):
    if isinstance(o, float):
        if o == 0:
            return 0
        if np.isinf(o) or np.isnan(o):
            return f"{o}"
        lo = int(log10(abs(o))) - (abs(o) > 10)
        if precision + lo < 0:
            return 0
        if precision * 2 < lo:
            return round(o)
        return round(o, precision - lo)
    if isinstance(o, dict):
        return {k: round_floats(v, precision) for k, v in o.items()}
    if isinstance(o, (list, tuple)):
        return [round_floats(x, precision) for x in o]
    return o


if False:
    with open("benchmark_production_OECM.json", "w") as f:
        json.dump(round_floats(production_bm), sort_keys=False, indent=2, fp=f)
    with open("benchmark_EI_OECM_S3.json", "w") as f:
        json.dump(round_floats(ei_bms[False]), sort_keys=False, indent=2, fp=f)
    with open("benchmark_EI_OECM_PC.json", "w") as f:
        json.dump(round_floats(ei_bms[True]), sort_keys=False, indent=2, fp=f)

In [ ]:
if False:
    production_index = pd.MultiIndex.from_tuples(
        [
            (
                production_bm[scope]["benchmarks"][bm]["sector"],
                production_bm[scope]["benchmarks"][bm]["region"],
                production_bm[scope]["benchmarks"][bm]["benchmark_metric"],
                scope,
                bm,
            )
            for scope in ["AnyScope"]
            for bm in range(len(production_bm[scope]["benchmarks"]))
        ],
        names=["sector", "region", "benchmark_metric", "scope", "bm_idx"],
    )
    df_production = pd.DataFrame.from_dict(
        {
            (idx[0], idx[1], idx[2], idx[3]): {
                projection["year"]: projection["value"]
                for projection in production_bm[idx[3]]["benchmarks"][idx[4]][
                    "projections_nounits"
                ]
            }
            for idx in production_index
        },
        orient="index",
    )
    df_production.index = production_index.droplevel("bm_idx")
    df_production.sort_index(inplace=True)

In [ ]:
benchmark_scopes = [["S1", "S2", "S1S2", "S3", "S1S2S3"], ["S1", "S2", "S1S2"]]

for wb_filename, production_centric in [
    ("benchmark_OECM_S3", False),
    ("benchmark_OECM_PC", True),
]:
    continue
    ei_index = pd.MultiIndex.from_tuples(
        [
            (
                ei_bms[production_centric][scope]["benchmarks"][bm]["sector"],
                ei_bms[production_centric][scope]["benchmarks"][bm]["region"],
                ei_bms[production_centric][scope]["benchmarks"][bm]["benchmark_metric"],
                scope,
                bm,
            )
            for scope in benchmark_scopes[production_centric]
            for bm in range(len(ei_bms[production_centric][scope]["benchmarks"]))
        ],
        names=["sector", "region", "benchmark_metric", "scope", "bm_idx"],
    )
    df_ei = pd.DataFrame.from_dict(
        {
            (idx[0], idx[1], idx[2], idx[3]): {
                projection["year"]: projection["value"]
                for projection in ei_bms[production_centric][idx[3]]["benchmarks"][
                    idx[4]
                ]["projections_nounits"]
            }
            for idx in ei_index
        },
        orient="index",
    )
    df_ei.index = ei_index.droplevel("bm_idx")
    df_ei.sort_index(inplace=True)

    wb_sheets = {"projected_production": df_production, "projected_ei": df_ei}

    with pd.ExcelWriter(f"{wb_filename}.xlsx") as writer:
        df_production.to_excel(writer, sheet_name="projected_production")
        df_ei.to_excel(writer, sheet_name="projected_ei")

In [ ]:
# wb = pd.read_excel('benchmark_EI_OECM_S3.xlsx', sheet_name=None)

In [ ]:
# wb['projected_ei'].fillna(method="ffill")